# Title here

In [53]:
# Import statements

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from torch import nn, optim
from torchsummary import summary
import time
from sklearn.metrics import r2_score

In [54]:
# Read data
data = pd.read_csv('final_test.csv')
print(data.shape)
data.head()

(119734, 4)


,weight,age,height,size
0,62,28.0,172.72,XL
1,59,36.0,167.64,L
2,61,34.0,165.10,M
3,65,27.0,175.26,L
4,62,45.0,172.72,M


In [22]:
print('Average height in the dataset: ',data.height.mean())
print('Average age in the dataset: ',data.age.mean())
print('Average weight in the dataset: ',data.weight.mean())
print('dtype of Height: ', data.height.dtype)
print('dtype of Age: ', data.age.dtype)
print('dtype of Weight: ', data.weight.dtype)

Average height in the dataset:  0.5126305346859098
Average age in the dataset:  0.2908317153485679
Average weight in the dataset:  0.3487439555341827
dtype of Height:  float64
dtype of Age:  float64
dtype of Weight:  float64


In [55]:
def map_size(size):
    if size == 'XXS':
        return 0.0
    elif size == 'S':
        return 1.0
    elif size == 'M':
        return 2.0
    elif size == 'L':
        return 3.0
    elif size == 'XL':
        return 4.0
    elif size == 'XXL':
        return 5.0
    elif size == 'XXXL':
        return 6.0

In [56]:
data = data.dropna()
data['weight'] = data['weight'].astype('float64')
data['size'] = data['size'].apply(map_size)


In [57]:
data.head()

,weight,age,height,size
0,62.0,28.0,172.72,4.0
1,59.0,36.0,167.64,3.0
2,61.0,34.0,165.10,2.0
3,65.0,27.0,175.26,3.0
4,62.0,45.0,172.72,2.0


In [58]:
def NormalizeData01(data, min_, max_):
    return (data - min_) / (max_ - min_)
def NormalizeData(data, min_, max_):
    return (2*((data - min_) / (max_ - min_))) - 1

data['weight'] = data['weight'].apply(lambda x: NormalizeData01(x, data['weight'].min(), data['weight'].max()))     
data['age'] = data['age'].apply(lambda x: NormalizeData01(x, data['age'].min(), data['age'].max()))      
data['height'] = data['height'].apply(lambda x: NormalizeData01(x, data['height'].min(), data['height'].max()))     



In [59]:
X = data[['weight', 'age', 'height']]
Y = data['size']
X = X.values
Y = Y.values

# DATASET 80-10-10 SPLIT PROPORTION TRAIN-VALIDATE-TEST RESPECTIVELY

X_train, X_val_test, Y_train, Y_val_test = train_test_split(X, Y, test_size=0.4, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_val_test, Y_val_test, test_size=0.5, random_state=42)

In [60]:
# CONVERTING DATA TO TENSORS

X_train = torch.from_numpy(X_train).type(torch.FloatTensor)

X_val = torch.from_numpy(X_val).type(torch.FloatTensor)

X_test = torch.from_numpy(X_test).type(torch.FloatTensor)

Y_train = torch.from_numpy(Y_train).type(torch.FloatTensor)

Y_val = torch.from_numpy(Y_val).type(torch.FloatTensor)

# SETTING TENSOR DATASETS

train_tensor = torch.utils.data.TensorDataset(X_train, Y_train)

validation_tensor = torch.utils.data.TensorDataset(X_val, Y_val)

test_tensor = torch.utils.data.TensorDataset(X_test)

# FINAL DATA LOADERS - SETTING BATCH CONFIGURATION

train_loader = torch.utils.data.DataLoader(train_tensor,
                                           batch_size=64,
                                           shuffle=True)

validation_loader = torch.utils.data.DataLoader(validation_tensor,
                                                batch_size=64,
                                                shuffle=False)

test_loader = torch.utils.data.DataLoader(test_tensor,
                                          batch_size=64,
                                          shuffle=False)


In [61]:
# MODEL SET-UP

input_size = 3
hidden_sizes = [64, 64]
output_size = 7
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.LeakyReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.LeakyReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      )

In [62]:
# XAVIER'S WEIGHTS INITIALIZATION

for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):
        torch.nn.init.xavier_uniform_(module.weight, gain=1.0)

In [66]:
# PRE-TRAINING SET-UP

optimizer = optim.Adam(model.parameters(), lr=0.01) # , momentum=0.9
epochs = 100
criterion = nn.CrossEntropyLoss()
model.to(device)
running_loss = 0
steps = 0
best_model_r2 = 0

In [67]:
# TRAINING START

print('Training Started!')
start = time.time()
for e in range(epochs):
    print('Epoch number: ', e + 1)

    for inputs, labels in train_loader:

        # TRAINING LOOP

        inputs, labels = inputs.float(), labels.float()
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.long())

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        steps += 1

        # 10 VALIDATION LOOPS PER EPOCH
        
        if steps == int(len(train_loader) / 5):
            sanity = []
            model.eval()
            accuracy = 0
            valid_loss = 0

            with torch.no_grad():

                for inputs, labels in validation_loader:

                    # VALIDATION LOOP

                    inputs, labels = inputs.float(), labels.float()
                    inputs, labels = inputs.to(device), labels.to(device)

                    outputs = model(inputs)

                    loss_valid = criterion(outputs, labels.long())
                    valid_loss += loss_valid.item()
                    
                    labels = labels.cpu()
                    outputs = outputs.cpu()
                    _, outputs = torch.max(outputs.data, 1)
                    
                    accuracy += r2_score(labels, outputs)
                    #top_p, top_class = outputs.topk(1, dim=1)
                    #equals = top_class == labels.view(*top_class.shape)
                    #accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
                overall_accuracy = (accuracy * 100) / len(validation_loader)
                if overall_accuracy > best_model_r2:
                    best_model_r2 = overall_accuracy
                    torch.save(model, 'test.pth')
                print(
                    f"Train loss: {running_loss / steps:.4f}.. "
                    f"Valid loss: {valid_loss / len(validation_loader):.4f}.. "
                    f"Valid accuracy: {(accuracy * 100) / len(validation_loader):.5f}..")

            running_loss = 0
            steps = 0
            model.train()

print('Training finished!')
print('Training lasted: ', time.time() - start)


Training Started!
Epoch number:  1
Train loss: 1.1517.. Valid loss: 1.1277.. Valid accuracy: 65.70854..
Train loss: 1.1370.. Valid loss: 1.1207.. Valid accuracy: 67.11798..
Train loss: 1.1384.. Valid loss: 1.1242.. Valid accuracy: 66.27736..
Train loss: 1.1416.. Valid loss: 1.1230.. Valid accuracy: 65.53367..
Train loss: 1.1321.. Valid loss: 1.1250.. Valid accuracy: 62.16697..
Epoch number:  2
Train loss: 1.1341.. Valid loss: 1.1274.. Valid accuracy: 67.16718..
Train loss: 1.1269.. Valid loss: 1.1362.. Valid accuracy: 63.39254..
Train loss: 1.1454.. Valid loss: 1.1291.. Valid accuracy: 63.79706..
Train loss: 1.1395.. Valid loss: 1.1264.. Valid accuracy: 64.61281..
Train loss: 1.1413.. Valid loss: 1.1336.. Valid accuracy: 64.41891..
Epoch number:  3
Train loss: 1.1260.. Valid loss: 1.1226.. Valid accuracy: 67.24946..
Train loss: 1.1370.. Valid loss: 1.1223.. Valid accuracy: 66.99579..
Train loss: 1.1390.. Valid loss: 1.1315.. Valid accuracy: 64.56005..
Train loss: 1.1421.. Valid loss: 1

Train loss: 1.1221.. Valid loss: 1.1359.. Valid accuracy: 63.77031..
Train loss: 1.1319.. Valid loss: 1.1186.. Valid accuracy: 65.57742..
Epoch number:  24
Train loss: 1.1459.. Valid loss: 1.1170.. Valid accuracy: 65.77211..
Train loss: 1.1156.. Valid loss: 1.1246.. Valid accuracy: 64.88984..
Train loss: 1.1398.. Valid loss: 1.1436.. Valid accuracy: 64.95418..
Train loss: 1.1310.. Valid loss: 1.1168.. Valid accuracy: 67.11958..
Train loss: 1.1289.. Valid loss: 1.1179.. Valid accuracy: 65.67761..
Epoch number:  25
Train loss: 1.1296.. Valid loss: 1.1184.. Valid accuracy: 67.31813..
Train loss: 1.1260.. Valid loss: 1.1165.. Valid accuracy: 65.95972..
Train loss: 1.1397.. Valid loss: 1.1172.. Valid accuracy: 66.19310..
Train loss: 1.1208.. Valid loss: 1.1478.. Valid accuracy: 66.01103..
Train loss: 1.1412.. Valid loss: 1.1335.. Valid accuracy: 65.39620..
Epoch number:  26
Train loss: 1.1250.. Valid loss: 1.1196.. Valid accuracy: 66.08140..
Train loss: 1.1330.. Valid loss: 1.1189.. Valid a

Train loss: 1.1409.. Valid loss: 1.1208.. Valid accuracy: 66.58538..
Train loss: 1.1350.. Valid loss: 1.1231.. Valid accuracy: 66.38629..
Train loss: 1.1160.. Valid loss: 1.1192.. Valid accuracy: 67.12821..
Train loss: 1.1320.. Valid loss: 1.1178.. Valid accuracy: 66.50809..
Epoch number:  47
Train loss: 1.1220.. Valid loss: 1.1224.. Valid accuracy: 65.66432..
Train loss: 1.1244.. Valid loss: 1.1240.. Valid accuracy: 66.67182..
Train loss: 1.1374.. Valid loss: 1.1268.. Valid accuracy: 64.69558..
Train loss: 1.1350.. Valid loss: 1.1172.. Valid accuracy: 66.76925..
Train loss: 1.1321.. Valid loss: 1.1222.. Valid accuracy: 64.98608..
Epoch number:  48
Train loss: 1.1246.. Valid loss: 1.1384.. Valid accuracy: 65.56486..
Train loss: 1.1321.. Valid loss: 1.1193.. Valid accuracy: 67.41871..
Train loss: 1.1215.. Valid loss: 1.1220.. Valid accuracy: 65.32540..
Train loss: 1.1178.. Valid loss: 1.1176.. Valid accuracy: 67.88013..
Train loss: 1.1334.. Valid loss: 1.1161.. Valid accuracy: 65.44335.

Train loss: 1.1322.. Valid loss: 1.1335.. Valid accuracy: 66.58444..
Epoch number:  69
Train loss: 1.1320.. Valid loss: 1.1224.. Valid accuracy: 66.41752..
Train loss: 1.1219.. Valid loss: 1.1199.. Valid accuracy: 65.51270..
Train loss: 1.1238.. Valid loss: 1.1205.. Valid accuracy: 66.72924..
Train loss: 1.1359.. Valid loss: 1.1172.. Valid accuracy: 66.61866..
Train loss: 1.1260.. Valid loss: 1.1249.. Valid accuracy: 64.72871..
Epoch number:  70
Train loss: 1.1378.. Valid loss: 1.1228.. Valid accuracy: 66.01771..
Train loss: 1.1331.. Valid loss: 1.1190.. Valid accuracy: 67.23282..
Train loss: 1.1244.. Valid loss: 1.1221.. Valid accuracy: 66.62639..
Train loss: 1.1262.. Valid loss: 1.1197.. Valid accuracy: 67.58004..
Train loss: 1.1264.. Valid loss: 1.1386.. Valid accuracy: 63.86933..
Epoch number:  71
Train loss: 1.1302.. Valid loss: 1.1255.. Valid accuracy: 67.37636..
Train loss: 1.1354.. Valid loss: 1.1264.. Valid accuracy: 65.73464..
Train loss: 1.1233.. Valid loss: 1.1211.. Valid a

Train loss: 1.1255.. Valid loss: 1.1197.. Valid accuracy: 65.24997..
Train loss: 1.1377.. Valid loss: 1.1230.. Valid accuracy: 66.95074..
Train loss: 1.1325.. Valid loss: 1.1227.. Valid accuracy: 67.18260..
Train loss: 1.1219.. Valid loss: 1.1277.. Valid accuracy: 65.93288..
Epoch number:  92
Train loss: 1.1199.. Valid loss: 1.1241.. Valid accuracy: 66.60893..
Train loss: 1.1276.. Valid loss: 1.1271.. Valid accuracy: 65.96050..
Train loss: 1.1254.. Valid loss: 1.1181.. Valid accuracy: 67.47132..
Train loss: 1.1374.. Valid loss: 1.1204.. Valid accuracy: 65.11596..
Train loss: 1.1271.. Valid loss: 1.1220.. Valid accuracy: 66.89883..
Epoch number:  93
Train loss: 1.1366.. Valid loss: 1.1249.. Valid accuracy: 65.38401..
Train loss: 1.1325.. Valid loss: 1.1430.. Valid accuracy: 63.10659..
Train loss: 1.1354.. Valid loss: 1.1194.. Valid accuracy: 65.26058..
Train loss: 1.1212.. Valid loss: 1.1214.. Valid accuracy: 66.75819..
Train loss: 1.1180.. Valid loss: 1.1214.. Valid accuracy: 66.12777.

In [ ]:
summary(model)

In [44]:
outputs = model(torch.Tensor([0.5,0.5,0.5]).to(device))
